In [152]:
import pandas as pd
from collections import Counter
# Load the datasets
transaction_data = pd.read_csv("transaction_data.csv")
purchase_data = pd.read_csv("purchase_behaviour.csv")

# Identify the top 3 most profitable products
top_products = (
    transaction_data.groupby(["PROD_NBR", "PROD_NAME"])["TOT_SALES"]
    .sum()
    .reset_index()
    .sort_values(by="TOT_SALES", ascending=False)
    # .head(3)
)

# Merge transaction data with customer data
merged_data = transaction_data.merge(purchase_data, on="LYLTY_CARD_NBR")

# Identify the most loyal customers based on total spending
loyal_customers = (
    merged_data.groupby(["LIFESTAGE", "PREMIUM_CUSTOMER"])["TOT_SALES"]
    .sum()
    .reset_index()
    .sort_values(by="TOT_SALES", ascending=False)
    # .head(3)  # The top segment
)

# Identify customer segments with highest purchase frequency
purchase_frequency = (
    merged_data.groupby(["LIFESTAGE", "PREMIUM_CUSTOMER"])['TXN_ID']
    .count()
    .reset_index()
    .sort_values(by="TXN_ID", ascending=False)
)

# Analyze average basket size per customer segment
average_basket_size = (
    merged_data.groupby(["LIFESTAGE", "PREMIUM_CUSTOMER"])['TOT_SALES']
    .mean()
    .reset_index()
    .sort_values(by="TOT_SALES", ascending=False)
)

# Identify best-selling products per customer segment
top_products_by_segment = (
    merged_data.groupby(["LIFESTAGE", "PREMIUM_CUSTOMER", "PROD_NAME"])["TOT_SALES"]
    .sum()
    .reset_index()
    .sort_values(by="TOT_SALES", ascending=False)
)

# Identify revenue by store
store_revenue = (
    transaction_data.groupby("STORE_NBR")["TOT_SALES"]
    .sum()
    .reset_index()
    .sort_values(by="TOT_SALES", ascending=False)
)
s={}
store_freq_dict=(
    dict(Counter(transaction_data["STORE_NBR"].to_list()))
)
strfrq=pd.DataFrame(store_freq_dict.items(),columns=["STR_NUMBR","VISITS"])
store_freq=(
    strfrq.sort_values(by="VISITS",ascending=False)
)
cust_most_visited=(
    store_freq["STR_NUMBR"].head(3)
)

First Insight: Top 3 Profitable Products

In [153]:
print("Top 3 Most Profitable Products:\n", top_products.head(3))
top_products.to_csv("profitable_products.csv")

Top 3 Most Profitable Products:
     PROD_NBR                                 PROD_NAME  TOT_SALES
3          4          Dorito Corn Chp     Supreme 380g    40352.0
13        14    Smiths Crnkle Chip  Orgnl Big Bag 380g    36367.6
15        16  Smiths Crinkle Chips Salt & Vinegar 330g    34804.2


Second Insight: Most Loyal Customer Segment and Customer Liftstyle by total sales

In [121]:
print("\nMost Loyal Customer Segment by sales:\n", loyal_customers.head(3))
loyal_customers.to_csv("loyal_customers_sales.csv")


Most Loyal Customer Segment by sales:
                 LIFESTAGE PREMIUM_CUSTOMER  TOT_SALES
6          OLDER FAMILIES           Budget  168363.25
19  YOUNG SINGLES/COUPLES       Mainstream  157621.60
13               RETIREES       Mainstream  155677.05


Third Insight: Top Products in Customer Segments

In [48]:
top_products_by_segment.to_csv("top_prod_segment.csv")
print("\nTop Products by Customer Segment:\n", top_products_by_segment.head(10))


Top Products by Customer Segment:
                   LIFESTAGE PREMIUM_CUSTOMER  \
2177  YOUNG SINGLES/COUPLES       Mainstream   
2252  YOUNG SINGLES/COUPLES       Mainstream   
2199  YOUNG SINGLES/COUPLES       Mainstream   
2243  YOUNG SINGLES/COUPLES       Mainstream   
761          OLDER FAMILIES           Budget   
2178  YOUNG SINGLES/COUPLES       Mainstream   
695          OLDER FAMILIES           Budget   
690          OLDER FAMILIES           Budget   
2172  YOUNG SINGLES/COUPLES       Mainstream   
2205  YOUNG SINGLES/COUPLES       Mainstream   

                                     PROD_NAME  TOT_SALES  
2177          Dorito Corn Chp     Supreme 380g    3659.50  
2252    Smiths Crnkle Chip  Orgnl Big Bag 380g    3516.40  
2199    Kettle Mozzarella   Basil & Pesto 175g    3380.40  
2243  Smiths Crinkle Chips Salt & Vinegar 330g    3317.40  
761   Smiths Crinkle Chips Salt & Vinegar 330g    3186.30  
2178          Doritos Cheese      Supreme 330g    3186.30  
695           D

Forth Insight: Customer segments and lifestyles that spend more per transaction, i.e higher Average Basket Size

In [49]:
average_basket_size.to_csv("avg_basket.csv")
print("\nAverage Basket Size per Customer Segment:\n", average_basket_size.head(10))


Average Basket Size per Customer Segment:
                  LIFESTAGE PREMIUM_CUSTOMER  TOT_SALES
1   MIDAGE SINGLES/COUPLES       Mainstream   7.647284
19   YOUNG SINGLES/COUPLES       Mainstream   7.558339
14                RETIREES          Premium   7.456174
11   OLDER SINGLES/COUPLES          Premium   7.449766
12                RETIREES           Budget   7.443445
9    OLDER SINGLES/COUPLES           Budget   7.430315
8           OLDER FAMILIES          Premium   7.322945
4             NEW FAMILIES       Mainstream   7.317806
3             NEW FAMILIES           Budget   7.297321
15          YOUNG FAMILIES           Budget   7.287201


Fifth Insight: Total Sales per Store

In [50]:
store_revenue.to_csv("revenue_per_store.csv")
print("\nRevenue by Store:\n", store_revenue.head(10))


Revenue by Store:
      STORE_NBR  TOT_SALES
225        226   18905.45
87          88   16333.25
164        165   15973.75
39          40   15559.50
236        237   15539.50
57          58   15251.45
198        199   14797.00
3            4   14647.65
202        203   14551.60
25          26   14469.30


Sixth Insight: Top most visited Stores

In [78]:
print("\nTop 3 most visited Stores:\n",store_freq.head(3))


Top 3 most visited Stores:
      STR_NUMBR  VISITS
102        226    2022
39          88    1873
166         93    1832


Seventh Insight: Most Frequent Customer Segments and Lifestyles in the top 3 most visited stores

In [132]:
mvstr=[]
store1={}
store2={}
store3={}
storepc1={}
storepc2={}
storepc3={}
mvstr=cust_most_visited.to_list()
for i in mvstr:
    # print(type(i))
    if i==226:
        store1=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"LIFESTAGE",]))
        storepc1=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"PREMIUM_CUSTOMER",]))
    elif i==88:
        store2=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"LIFESTAGE"]))
        storepc2=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"PREMIUM_CUSTOMER",]))
    else:
        store3=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"LIFESTAGE"]))
        storepc3=dict(Counter(merged_data.loc[merged_data["STORE_NBR"]==i,"PREMIUM_CUSTOMER",]))
# print(store1)
print("Most Frequent Customer Lifestyle at the highest visited store:\n"+str(max(store1,key=store1.get)))
print("Most Frequent Customer Segment at the highest visited store:\n"+str(max(storepc1,key=storepc1.get)))
print("Most Frequent Customer Lifestyle at the second visited store:\n"+str(max(store2,key=store1.get)))
print("Most Frequent Customer Segment at the second highest visited store:\n"+str(max(storepc2,key=storepc2.get)))
print("Most Frequent Customer Lifestyle at the third visited store:\n"+str(max(store3,key=store1.get)))
print("Most Frequent Customer Segment at the Third highest visited store:\n"+str(max(storepc3,key=storepc3.get)))

Most Frequent Customer Lifestyle at the highest visited store:
OLDER SINGLES/COUPLES
Most Frequent Customer Segment at the highest visited store:
Mainstream
Most Frequent Customer Lifestyle at the second visited store:
OLDER SINGLES/COUPLES
Most Frequent Customer Segment at the second highest visited store:
Mainstream
Most Frequent Customer Lifestyle at the third visited store:
OLDER SINGLES/COUPLES
Most Frequent Customer Segment at the Third highest visited store:
Budget


Eigth Insight: Top 5 Customer Segment and Lifestyle in terms of product quantity

In [151]:
quant=(merged_data.groupby(["LIFESTAGE","PREMIUM_CUSTOMER"])["PROD_QTY"]).count().reset_index().sort_values(by="PROD_QTY", ascending=False)
print("Top 5 Customer Segment and Lifestyle in terms of product quantity purchased:\n",quant.head())

Top 5 Customer Segment and Lifestyle in terms of product quantity purchased:
                 LIFESTAGE PREMIUM_CUSTOMER  PROD_QTY
6          OLDER FAMILIES           Budget     23160
13               RETIREES       Mainstream     21466
19  YOUNG SINGLES/COUPLES       Mainstream     20854
15         YOUNG FAMILIES           Budget     19122
9   OLDER SINGLES/COUPLES           Budget     18407


My Insights:
Key Indicators of loyalty include total sales, top products by customer segments, money spent per transaction (on average) AKA Average Basket Size, Most frequent Customer Segment in the most visited stores.

My Hypothesis:
1) Older Families in the budget segment have the higher contribution to total sales due to their relatively higher family members and tendency to save money. This is likely why the goods they have purchased are in higher quantity but not higher diversity.
2) Top 3 products in sale: Dorito Corn Chp Supreme 380g, Smiths Crnkle Chip  Orgnl Big Bag 380g, Smiths Crinkle Chips Salt & Vinegar 330g are all bigger quantities of popular chips brands. Their larger quantity and positive customer perception are likely factors behind their large sales
3) The average basket size of MIDAGE SINGLES/COUPLES in mainstream segment, YOUNG SINGLES/COUPLES in mainstream segment, and RETIREES in the premium segment is higher than the rest. This indicates that customers that belong to these lifestyles and segments have higher transactions per visit. However, as seen in insight 7, these lifestyles don't show up in high frequencies at the most visited stores.
4) The Most frequent customers at the top 3 most visited stores belong to OLDER SINGLES/COUPLES and particularly the Mainstream segment. This information coupled with the top selling products in each customer segment should give us confidence to increase the stock of or run discount campaigns on the products preferred by OLDER SINGLES/COUPLES in these particular stores.